# Bundling a MONAI Train Bundle
It is simple to package your MONAI code into a MONAI Application. For this tutorial, we will convert a Clara 4.1 model from NGC
- For imports, all we need is the json package

In [1]:
import json

- We'll need to create our bundle directory

In [2]:
# Create app directory structure
APP_ROOT = "/workspace/monai-bundle-demo-covid19-lung-segmentation"
!mkdir -p $APP_ROOT
!mkdir -p $APP_ROOT/configs
!mkdir -p $APP_ROOT/docs

- At the top of our json, we declare our imports for the bundle
- Next we set a few variables
- Anything that begins with an @ is a variable that is referenced elsewhere in the config file.
- Anything that begins with a $ is evaluated before being written into the code.

In [3]:
# Common elements
bundle_config = {
    "imports": ["$import glob", "$import os", "$import ignite"],
    "device": "$torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')",
    "bundle_root": APP_ROOT,
    "ckpt_dir": "$@bundle_root + '/models'",
    "output_dir": "$@bundle_root + '/eval'",
    "dataset_dir": "/workspace/datasets",
    "images": "$list(sorted(glob.glob(@dataset_dir + '/CT-Covid-19-August2020/*.nii.gz')))",
    "labels": "$list(sorted(glob.glob(@dataset_dir + '/CT-Covid-19-August2020-seg/*.nii.gz')))",
    "val_interval": 5,
    "epochs": 100,
}

- Next we download the Clara 4.1 app and extract it.
- In this tutorial, we downloaded and extracted the following:
- https://catalog.ngc.nvidia.com/orgs/nvidia/teams/med/models/clara_pt_covid19_ct_lung_segmentation

In [4]:
CLARA_APP_ROOT = "/workspace/covid19_seg"

- We'll run this tutorial on the train configuration.

In [5]:
# Read CLARA 4.1 Training Config
with open(CLARA_APP_ROOT + "/config/config_train.json") as f:
    config_train = json.load(f)

- Here we check the model in the clara app, an define the model in the MONAI Bundle style

In [6]:
# Get model
config_train["train"]["model"]

{'name': 'AHNet',
 'args': {'spatial_dims': 3,
  'in_channels': '{in_channels}',
  'out_channels': '{out_classes}',
  'psp_block_num': 0,
  'upsample_mode': 'trilinear',
  'pretrained': True}}

In [7]:
# Convert variables and change to network_def
network_def = json.loads(
    json.dumps(config_train["train"]["model"])
    .replace('"{in_channels}"', str(config_train["in_channels"]))
    .replace('"{out_classes}"', str(config_train["out_classes"]))
)
network_def = {"_target_": network_def["name"], **network_def["args"]}
network_def

{'_target_': 'AHNet',
 'spatial_dims': 3,
 'in_channels': 1,
 'out_channels': 2,
 'psp_block_num': 0,
 'upsample_mode': 'trilinear',
 'pretrained': True}

In [8]:
# Add network to bundle config
bundle_config["network_def"] = network_def
bundle_config["network"] = "$@network_def.to(@device)"

- Next we do the same with the loss and optimizer

In [9]:
# Check out Loss and Optimizer
print(config_train["train"]["loss"])
print(config_train["train"]["optimizer"])

{'name': 'DiceLoss', 'args': {'jaccard': False, 'to_onehot_y': True, 'softmax': True, 'squared_pred': True}}
{'name': 'Adam', 'args': {'params': '#@model.parameters()', 'lr': '{learning_rate}'}}


In [10]:
# Define our target loss and optimizer
loss = {
    "_target_": config_train["train"]["loss"]["name"],
    **config_train["train"]["loss"]["args"],
}
optimizer = {"_target_": "torch.optim.Adam"}
optimizer["params"] = "$@network.parameters()"  # Convert to MONAI Bundle schema
optimizer["lr"] = config_train["learning_rate"]
print(loss)
print(optimizer)

{'_target_': 'DiceLoss', 'jaccard': False, 'to_onehot_y': True, 'softmax': True, 'squared_pred': True}
{'_target_': 'torch.optim.Adam', 'params': '$@network.parameters()', 'lr': 0.0001}


In [11]:
# Add loss and optimizer to bundle config
bundle_config["loss"] = loss
bundle_config["optimizer"] = optimizer

- Next let's set up the configs that are specific to training.
- We'll wrap them under a train header.

In [12]:
# Set up training configs
train = {}

- Start with our preprocessing

In [13]:
# Read Pre-Transforms
pre_transforms = config_train["train"]["pre_transforms"]
for t in pre_transforms:
    print(f"""{t["name"]}({", ".join([f'''{k}={str(v)}''' for k,v in t["args"].items()])})""")

LoadImaged(keys=['image', 'label'])
EnsureChannelFirstd(keys=['image', 'label'])
Spacingd(keys=['image', 'label'], pixdim=[0.8, 0.8, 5], mode=['bilinear', 'nearest'], align_corners=[True, True])
ScaleIntensityRanged(keys=image, a_min=-1500, a_max=500, b_min=0, b_max=1, clip=True)
CropForegroundd(keys=['image', 'label'], source_key=image)
SpatialPadd(keys=['image', 'label'], spatial_size=['{roi_x}', '{roi_y}', '{roi_z}'], mode=['minimum', 'constant'])
RandCropByPosNegLabeld(keys=['image', 'label'], label_key=label, spatial_size=['{roi_x}', '{roi_y}', '{roi_z}'], pos=1, neg=1, num_samples=2, image_key=image, image_threshold=0)
RandFlipd(keys=['image', 'label'], spatial_axis=[0], prob=0.1)
RandFlipd(keys=['image', 'label'], spatial_axis=[1], prob=0.1)
RandFlipd(keys=['image', 'label'], spatial_axis=[2], prob=0.1)
RandRotate90d(keys=['image', 'label'], prob=0.1, max_k=3)
RandShiftIntensityd(keys=image, offsets=0.1, prob=0.5)
ToTensord(keys=['image', 'label'])


- Some variables are being referenced in this application.
- Here I've set the values for the variables directly into the functions.
- Another method is to convert `{roi_x}` to `@roi_x` and assign the variable at the top of the bundle config.
- We also remove the random transforms for the validation transforms, which we will reference at the validation section.

In [14]:
# Replace Variables
pre_transforms = json.loads(
    json.dumps(config_train["train"]["pre_transforms"])
    .replace('"{roi_x}"', str(config_train["roi_x"]))
    .replace('"{roi_y}"', str(config_train["roi_y"]))
    .replace('"{roi_z}"', str(config_train["roi_z"]))
)
train_transforms = {
    "_target_": "Compose",
    "transforms": [{"_target_": t["name"], **t["args"]} for t in pre_transforms],
}
validation_transforms = {
    "_target_": "Compose",
    "transforms": [
        {"_target_": t["name"], **t["args"]}
        for t in pre_transforms
        if "Rand" not in t["name"]
    ],
}
print(train_transforms)
print(validation_transforms)

{'_target_': 'Compose', 'transforms': [{'_target_': 'LoadImaged', 'keys': ['image', 'label']}, {'_target_': 'EnsureChannelFirstd', 'keys': ['image', 'label']}, {'_target_': 'Spacingd', 'keys': ['image', 'label'], 'pixdim': [0.8, 0.8, 5], 'mode': ['bilinear', 'nearest'], 'align_corners': [True, True]}, {'_target_': 'ScaleIntensityRanged', 'keys': 'image', 'a_min': -1500, 'a_max': 500, 'b_min': 0, 'b_max': 1, 'clip': True}, {'_target_': 'CropForegroundd', 'keys': ['image', 'label'], 'source_key': 'image'}, {'_target_': 'SpatialPadd', 'keys': ['image', 'label'], 'spatial_size': [224, 224, 32], 'mode': ['minimum', 'constant']}, {'_target_': 'RandCropByPosNegLabeld', 'keys': ['image', 'label'], 'label_key': 'label', 'spatial_size': [224, 224, 32], 'pos': 1, 'neg': 1, 'num_samples': 2, 'image_key': 'image', 'image_threshold': 0}, {'_target_': 'RandFlipd', 'keys': ['image', 'label'], 'spatial_axis': [0], 'prob': 0.1}, {'_target_': 'RandFlipd', 'keys': ['image', 'label'], 'spatial_axis': [1], 

In [15]:
# Add train pre-transforms
train["pre_transforms"] = train_transforms

- Next we convert the training dataset and dataloader as well

In [16]:
# Get Dataset and DataLoader
print(config_train["train"]["dataset"])
print(config_train["train"]["dataloader"])

{'name': 'CacheDataset', 'data_list_file_path': '{DATASET_JSON}', 'data_file_base_dir': '{DATA_ROOT}', 'data_list_key': '{TRAIN_DATALIST_KEY}', 'args': {'transform': '@pre_transforms', 'cache_rate': 1, 'num_workers': 4}}
{'name': 'DataLoader', 'args': {'dataset': '@dataset', 'batch_size': 2, 'shuffle': True, 'num_workers': 4}}


- Here we use the first 90% of the images as our training images.
- Notice that we can write code to be evaluated directly in the config

In [17]:
dataset = {
    "_target_": config_train["train"]["dataset"]["name"],
    **config_train["train"]["dataset"]["args"]
}
dataset["data"] = "$[{'image': i, 'label': l} for i, l in zip(@images[-9:], @labels[-9:])]"
dataset["transform"] = "@train#pre_transforms"
dataloader = {
    "_target_": config_train["train"]["dataloader"]["name"],
    **config_train["train"]["dataloader"]["args"]
}
dataloader["dataset"] = "@train#dataset"
print(dataset)
print(dataloader)

{'_target_': 'CacheDataset', 'transform': '@train#pre_transforms', 'cache_rate': 1, 'num_workers': 4, 'data': "$[{'image': i, 'label': l} for i, l in zip(@images[-9:], @labels[-9:])]"}
{'_target_': 'DataLoader', 'dataset': '@train#dataset', 'batch_size': 2, 'shuffle': True, 'num_workers': 4}


In [18]:
# Add train dataset and dataloader
train["dataset"] = dataset
train["dataloader"] = dataloader

- Next we set up the inferer to use for training

In [19]:
# Inferer
inferer = {"_target_": config_train["train"]["inferer"]["name"]}
print(inferer)

{'_target_': 'SimpleInferer'}


In [20]:
# Add train inferer
train["inferer"] = inferer

- Next we convert the learning rate scheduler as well

In [21]:
# LR_Scheduler
print(config_train["train"]["lr_scheduler"])

{'name': 'StepLR', 'args': {'optimizer': '@optimizer', 'step_size': 5000, 'gamma': 0.1}}


In [22]:
lr_scheduler = {
    "_target_": "torch.optim.lr_scheduler.StepLR",
    **config_train["train"]["lr_scheduler"]["args"]
}
lr_scheduler["optimizer"] = "@optimizer"
print(lr_scheduler)

{'_target_': 'torch.optim.lr_scheduler.StepLR', 'optimizer': '@optimizer', 'step_size': 5000, 'gamma': 0.1}


In [23]:
# Add train lr_scheduler
train["lr_scheduler"] = lr_scheduler

- Next let's take a look at the handlers

In [24]:
[print(handler) for handler in config_train["train"]["handlers"]]

{'name': 'CheckpointLoader', 'disabled': '{dont_load_ckpt_model}', 'args': {'load_path': '{MMAR_CKPT}', 'load_dict': {'model': '@model'}}}
{'name': 'LrScheduleHandler', 'args': {'lr_scheduler': '@lr_scheduler', 'print_lr': True}}
{'name': 'ValidationHandler', 'args': {'validator': '@evaluator', 'epoch_level': True, 'interval': '{num_interval_per_valid}'}}
{'name': 'CheckpointSaver', 'rank': 0, 'args': {'save_dir': '{MMAR_CKPT_DIR}', 'save_dict': {'model': '@model', 'optimizer': '@optimizer', 'lr_scheduler': '@lr_scheduler', 'train_conf': '@conf'}, 'save_final': True, 'save_interval': 400}}
{'name': 'StatsHandler', 'rank': 0, 'args': {'tag_name': 'train_loss', 'output_transform': "#monai.handlers.from_engine(['loss'], first=True)"}}
{'name': 'TensorBoardStatsHandler', 'rank': 0, 'args': {'log_dir': '{MMAR_CKPT_DIR}', 'tag_name': 'train_loss', 'output_transform': "#monai.handlers.from_engine(['loss'], first=True)"}}


[None, None, None, None, None, None]

Since the handlers are more complicated, we'll write them one by one.

In [25]:
handlers = [
    {
        "_target_": "LrScheduleHandler",
        "lr_scheduler": "@train#lr_scheduler",
        "print_lr": True,
    },
    {
        "_target_": "ValidationHandler",
        "validator": "@validate#evaluator",
        "epoch_level": True,
        "interval": "@val_interval",
    },
    {
        "_target_": "CheckpointSaver",
        "save_dir": "@ckpt_dir",
        "save_dict": {
            "model": "@network",
            "optimizer": "@optimizer",
            "lr_scheduler": "@train#lr_scheduler",
        },
        "save_final": True,
        "save_interval": 400,
    },
    {
        "_target_": "StatsHandler",
        "tag_name": "train_loss",
        "output_transform": "$monai.handlers.from_engine(['loss'], first=True)",
    },
    {
        "_target_": "TensorBoardStatsHandler",
        "log_dir": "@output_dir",
        "tag_name": "train_loss",
        "output_transform": "$monai.handlers.from_engine(['loss'], first=True)",
    },
]
[print(handler) for handler in handlers]


{'_target_': 'LrScheduleHandler', 'lr_scheduler': '@train#lr_scheduler', 'print_lr': True}
{'_target_': 'ValidationHandler', 'validator': '@validate#evaluator', 'epoch_level': True, 'interval': '@val_interval'}
{'_target_': 'CheckpointSaver', 'save_dir': '@ckpt_dir', 'save_dict': {'model': '@network', 'optimizer': '@optimizer', 'lr_scheduler': '@train#lr_scheduler'}, 'save_final': True, 'save_interval': 400}
{'_target_': 'StatsHandler', 'tag_name': 'train_loss', 'output_transform': "$monai.handlers.from_engine(['loss'], first=True)"}
{'_target_': 'TensorBoardStatsHandler', 'log_dir': '@output_dir', 'tag_name': 'train_loss', 'output_transform': "$monai.handlers.from_engine(['loss'], first=True)"}


[None, None, None, None, None]

In [26]:
# Add train handlers
train["handlers"] = handlers

- Next we have the postprocessing transforms

In [27]:
config_train["train"]["post_transforms"]

[{'name': 'Activationsd', 'args': {'keys': 'pred', 'softmax': True}},
 {'name': 'AsDiscreted',
  'args': {'keys': ['pred', 'label'],
   'argmax': [True, False],
   'to_onehot': True,
   'n_classes': '{out_classes}'}}]

In [28]:
post_transforms = [
    {
        "_target_": t["name"],
        **{
            k: config_train["out_classes"] if k == "to_onehot" else v
            for k, v in t["args"].items()
            if k != "n_classes"
        },
    }
    for t in config_train["train"]["post_transforms"]
]
post_transforms = {
    "_target_": "Compose",
    "transforms": post_transforms,
}

In [29]:
# Add train post_transforms
train["post_transforms"] = post_transforms

- Then we convert our key metric

In [30]:
config_train["train"]["key_metric"]

{'name': 'Accuracy',
 'log_label': 'train_acc',
 'args': {'output_transform': "#monai.handlers.from_engine(['pred', 'label'])"}}

- Note that we'll need to address the Accuracy metric with ignite.metrics

In [31]:
key_metric = {
    "_target_": "ignite.metrics.Accuracy",
    "output_transform": "$monai.handlers.from_engine(['pred', 'label'])",
}
key_metric

{'_target_': 'ignite.metrics.Accuracy',
 'output_transform': "$monai.handlers.from_engine(['pred', 'label'])"}

In [32]:
# Add train key_metric
train["key_metric"] = {"train_accuracy": key_metric}

- Finally we can create our SupervisedTrainer for ignite to handle our training

In [33]:
config_train["train"]["trainer"]

{'name': 'SupervisedTrainer',
 'args': {'max_epochs': '{epochs}',
  'device': 'cuda',
  'train_data_loader': '@dataloader',
  'network': '@model',
  'loss_function': '@loss',
  'optimizer': '@optimizer',
  'inferer': '@inferer',
  'postprocessing': '@post_transforms',
  'key_train_metric': '@key_metric',
  'train_handlers': '@handlers',
  'amp': '{amp}'}}

- Note that we refer to all our variables that are referred to in train by prefixing with `train#`

In [34]:
trainer = {
    "_target_": config_train["train"]["trainer"]["name"],
    "max_epochs": "@epochs",
    "device": "@device",
    "train_data_loader": "@train#dataloader",
    "network": "@network",
    "loss_function": "@loss",
    "optimizer": "@optimizer",
    "inferer": "@train#inferer",
    "postprocessing": "@train#post_transforms",
    "key_train_metric": "@train#key_metric",
    "train_handlers": "@train#handlers",
    "amp": True,
}
trainer

{'_target_': 'SupervisedTrainer',
 'max_epochs': '@epochs',
 'device': '@device',
 'train_data_loader': '@train#dataloader',
 'network': '@network',
 'loss_function': '@loss',
 'optimizer': '@optimizer',
 'inferer': '@train#inferer',
 'postprocessing': '@train#post_transforms',
 'key_train_metric': '@train#key_metric',
 'train_handlers': '@train#handlers',
 'amp': True}

In [35]:
# add to our configs
train["trainer"] = trainer
bundle_config["train"] = train

- Validation section is done similarly

In [36]:
validate = {}
validate["pre_transforms"] = validation_transforms
validate["dataset"] = {
    "_target_": "CacheDataset",
    "data": "$[{'image': i, 'label': l} for i, l in zip(@images[-9:], @labels[-9:])]",
    "transform": "@validate#pre_transforms",
    "cache_rate": 1.0,
    "num_workers": 2,
}
validate["dataloader"] = {
    "_target_": "DataLoader",
    "dataset": "@validate#dataset",
    "batch_size": 1,
    "shuffle": False,
    "num_workers": 2,
}

In [37]:
config_train["validate"]["inferer"]

{'name': 'SlidingWindowInferer',
 'args': {'roi_size': ['{roi_x}', '{roi_y}', '{roi_z}'],
  'sw_batch_size': 4,
  'overlap': 0.5}}

In [38]:
inferer = json.loads(
    json.dumps(config_train["validate"]["inferer"])
    .replace('"{roi_x}"', str(config_train["roi_x"]))
    .replace('"{roi_y}"', str(config_train["roi_y"]))
    .replace('"{roi_z}"', str(config_train["roi_z"]))
)
inferer = {
    "_target_": inferer["name"],
    **inferer["args"],
}

In [39]:
validate["inferer"] = inferer

In [40]:
config_train["validate"]["handlers"]

[{'name': 'StatsHandler',
  'rank': 0,
  'args': {'output_transform': 'lambda x: None'}},
 {'name': 'TensorBoardStatsHandler',
  'rank': 0,
  'args': {'log_dir': '{MMAR_CKPT_DIR}',
   'output_transform': 'lambda x: None'}},
 {'name': 'CheckpointSaver',
  'rank': 0,
  'args': {'save_dir': '{MMAR_CKPT_DIR}',
   'save_dict': {'model': '@model', 'train_conf': '@conf'},
   'save_key_metric': True}}]

In [41]:
validate["handlers"] = [
    {"_target_": "StatsHandler", "output_transform": "$lambda x: None"},
    {
        "_target_": "TensorBoardStatsHandler",
        "log_dir": "@output_dir",
        "output_transform": "$lambda x: None",
    },
    {
        "_target_": "CheckpointSaver",
        "save_dir": "@ckpt_dir",
        "save_dict": {"model": "@network"},
        "save_key_metric": True,
        "key_metric_filename": "val_model.pt"
    },
]
validate["handlers"]

[{'_target_': 'StatsHandler', 'output_transform': '$lambda x: None'},
 {'_target_': 'TensorBoardStatsHandler',
  'log_dir': '@output_dir',
  'output_transform': '$lambda x: None'},
 {'_target_': 'CheckpointSaver',
  'save_dir': '@ckpt_dir',
  'save_dict': {'model': '@network'},
  'save_key_metric': True,
  'key_metric_filename': 'val_model.pt'}]

In [42]:
config_train["validate"]["key_metric"]

{'name': 'MeanDice',
 'log_label': 'val_mean_dice',
 'args': {'include_background': False,
  'output_transform': "#monai.handlers.from_engine(['pred', 'label'])"}}

In [43]:
validate["key_metric"] = {
    "val_mean_dice": {
        "_target_": config_train["validate"]["key_metric"]["name"],
        **config_train["validate"]["key_metric"]["args"],
    }
}
validate["key_metric"]["val_mean_dice"][
    "output_transform"
] = "$monai.handlers.from_engine(['pred', 'label'])"
validate["key_metric"]

{'val_mean_dice': {'_target_': 'MeanDice',
  'include_background': False,
  'output_transform': "$monai.handlers.from_engine(['pred', 'label'])"}}

In [44]:
config_train["validate"]["additional_metrics"]

[{'name': 'Accuracy',
  'log_label': 'val_acc',
  'args': {'output_transform': "#monai.handlers.from_engine(['pred', 'label'])"}}]

In [45]:
validate["additional_metrics"] = {
    "val_accuracy": {
        "_target_": "ignite.metrics.Accuracy",
        "output_transform": "$monai.handlers.from_engine(['pred', 'label'])",
    }
}
validate["additional_metrics"]

{'val_accuracy': {'_target_': 'ignite.metrics.Accuracy',
  'output_transform': "$monai.handlers.from_engine(['pred', 'label'])"}}

In [46]:
config_train["validate"]["evaluator"]

{'name': 'SupervisedEvaluator',
 'args': {'device': 'cuda',
  'val_data_loader': '@dataloader',
  'network': '@model',
  'inferer': '@inferer',
  'postprocessing': '@post_transforms',
  'key_val_metric': '@key_metric',
  'additional_metrics': '@additional_metrics',
  'val_handlers': '@handlers',
  'amp': '{amp}'}}

In [47]:
evaluator = {
    "_target_": config_train["validate"]["evaluator"]["name"],
    "device": "@device",
    "val_data_loader": "@validate#dataloader",
    "network": "@network",
    "inferer": "@validate#inferer",
    "postprocessing": "@train#post_transforms",
    "key_val_metric": "@validate#key_metric",
    "additional_metrics": "@validate#additional_metrics",
    "val_handlers": "@validate#handlers",
    "amp": True,
}
evaluator

{'_target_': 'SupervisedEvaluator',
 'device': '@device',
 'val_data_loader': '@validate#dataloader',
 'network': '@network',
 'inferer': '@validate#inferer',
 'postprocessing': '@train#post_transforms',
 'key_val_metric': '@validate#key_metric',
 'additional_metrics': '@validate#additional_metrics',
 'val_handlers': '@validate#handlers',
 'amp': True}

In [48]:
validate["evaluator"] = evaluator

In [49]:
bundle_config["validate"] = validate

- Finally, we add a "training" section
- This includes the code that we will run if we select "training"

In [50]:
bundle_config["training"] = [
    "$monai.utils.set_determinism(seed=123)",
    "$setattr(torch.backends.cudnn, 'benchmark', True)",
    "$@train#trainer.run()"
]

- This is what the overall bundle_config will look like

In [51]:
bundle_config

{'imports': ['$import glob', '$import os', '$import ignite'],
 'device': "$torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')",
 'bundle_root': '/workspace/monai-bundle-demo-covid19-lung-segmentation',
 'ckpt_dir': "$@bundle_root + '/models'",
 'output_dir': "$@bundle_root + '/eval'",
 'dataset_dir': '/workspace/datasets',
 'images': "$list(sorted(glob.glob(@dataset_dir + '/CT-Covid-19-August2020/*.nii.gz')))",
 'labels': "$list(sorted(glob.glob(@dataset_dir + '/CT-Covid-19-August2020-seg/*.nii.gz')))",
 'val_interval': 5,
 'epochs': 100,
 'network_def': {'_target_': 'AHNet',
  'spatial_dims': 3,
  'in_channels': 1,
  'out_channels': 2,
  'psp_block_num': 0,
  'upsample_mode': 'trilinear',
  'pretrained': True},
 'network': '$@network_def.to(@device)',
 'loss': {'_target_': 'DiceLoss',
  'jaccard': False,
  'to_onehot_y': True,
  'softmax': True,
  'squared_pred': True},
 'optimizer': {'_target_': 'torch.optim.Adam',
  'params': '$@network.parameters()',
  'lr': 0.0001},
 '

- Next we'll open up a train.json file and write the config into it.

In [52]:
with open(APP_ROOT + "/configs/train.json", "w") as f:
    json.dump(bundle_config, f)

- Finally to test our code, we can use `monai.bundle run`
- then we specify `training` which will run the code under the training section that we wrote at the end
- and we specify the config file to be the one that we just wrote into

In [53]:
!python -m monai.bundle run training --config_file $APP_ROOT/configs/train.json

2022-11-29 06:22:51,125 - INFO - --- input summary of monai.bundle.scripts.run ---
2022-11-29 06:22:51,125 - INFO - > runner_id: 'training'
2022-11-29 06:22:51,125 - INFO - > config_file: '/workspace/monai-bundle-demo-covid19-lung-segmentation/configs/train.json'
2022-11-29 06:22:51,125 - INFO - ---


Loading dataset: 100%|████████████████████████████| 9/9 [00:05<00:00,  1.56it/s]
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████████████████████████████████| 97.8M/97.8M [00:02<00:00, 39.7MB/s]
Loading dataset: 100%|████████████████████████████| 9/9 [00:07<00:00,  1.24it/s]
2022-11-29 06:23:08,975 - Engine run resuming from iteration 0, epoch 0 until 100 epochs
2022-11-29 06:23:14,657 - Epoch: 1/100, Iter: 1/5 -- train_loss: 0.5739 
2022-11-29 06:23:14,887 - Epoch: 1/100, Iter: 2/5 -- train_loss: 0.5293 
2022-11-29 06:23:15,113 - Epoch: 1/100, Iter: 3/5 -- train_loss: 0.5801 
2022-11-29 06